Aprendizaje Automático · Grado en Ingeniería Informática · Curso 2022/23
---
Práctica 1: Predicción de la producción de energía solar
<br><br>
**Cuaderno 2 - Predicciones con datos de competición**
<br><br>
***Grupo Reducido 82 - Grupo de laboratorio 13***

# Importación de librerías y datos

Importamos las datos necesarios y conectamos google drive

In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


# Cargamos los datos de entrenamiento y competición.
# Datos disponibles.
df = pd.read_csv('wind_ava.csv')

# Datos competición.
comp_df = pd.read_csv('wind_comp.csv')

# Elimina todos los campos que no terminen en 13
columnas = df.columns[df.columns.str.endswith('13') | (df.columns == 'energy')]
df = df[columnas]

# Elimina todos los campos que no terminen en 13
columnas = comp_df.columns[comp_df.columns.str.endswith('13') | (comp_df.columns == 'energy')]
col_datetime = comp_df.columns[comp_df.columns == 'datetime']
df_datetime = comp_df[col_datetime]
comp_df = comp_df[columnas]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importamos las librerías necesarias

In [3]:
# Importaciones necesarias.
import numpy as np
import pandas as pd
import pickle
from rich import print
from sklearn.preprocessing import StandardScaler


In [4]:

# Semilla
np.random.seed(100472315)

# Datos.
X = df.drop(['energy'], axis=1)

# Etiquetas.
y = df['energy']

# Enntrenar el modelo con todos los datos disponibles y predecimos los datos de competición.
X_train = X
y_train = y
X_test = comp_df

# TODO: poner nuestro método de escalado
scaler = StandardScaler()
scaler.fit(X_train)

X_train_n = scaler.transform(X_train)
X_test_n = scaler.transform(X_test)

# TODO: poner nuestro método de escalado
scaler = StandardScaler()
scaler.fit(y_train.values.reshape(-1, 1))
y_train_n = scaler.transform(y_train.values.reshape(-1, 1))

# Entrenamiento modelo final
Entrenamos los datos utilizando el modelo XXX y guardamos el modelo final para utilizarlo con los datos de competición.
MEJORES HIPERPARAMETROS PARA MODELO ELEGIDO (KNN):

{'weights': 'distance', 'p': 1, 'n_neighbors': 20, 'metric': 'minkowski', 'leaf_size': 81}

In [5]:
from sklearn.neighbors import KNeighborsRegressor
# Entrenamos el modelo
knn = KNeighborsRegressor(weights='distance', p=1, n_neighbors=20, metric='minkowski', leaf_size=81)

knn.fit(X_train, y_train)
knn.fit(X_train_n, y_train_n)

# Guardamos el modelo final en un archivo pkl
with open("modelo_final.pkl", "wb") as f:
    pickle.dump(knn, f)

# Predicciones

Para obtener las predicciones, utilizaremos el modelo final previamente entrenado y guardaremos los resultados en el fichero *predicciones.csv*



In [6]:
y_pred_n = knn.predict(X_test_n)
y_pred = scaler.inverse_transform(y_pred_n)

# Truncamos los valores negativos a 0.
y_pred[y_pred < 0] = 0
# Guardamos las predicciones en el fichero predicciones.csv
predicciones_df = pd.DataFrame(y_pred, index=X_test.index, columns=["salida"])
predicciones_df.to_csv("predicciones_salida.csv")

Finalmente, imprimimos los resultados que hemos obtenido y guardado en nuestro csv predicciones

In [7]:
print(predicciones_df)

salida
0      566.448627
1      317.951280
2     1257.514052
3      518.152013
4      568.118041
...           ...
1184   280.980845
1185   195.742638
1186   215.866280
1187   164.523398
1188   129.866575

[1189 rows x 1 columns]


Guardaremos también, en un csv diferente, los datos de salida y el datetime.

In [8]:
columna_datetime = df_datetime.loc[X_test.index]
predicciones2_df = pd.concat([columna_datetime, predicciones_df['salida']], axis=1)

predicciones2_df.to_csv("predicciones.csv")
print(predicciones2_df)

datetime       salida
0     2010-01-01 00:00:00   566.448627
1     2010-01-01 06:00:00   317.951280
2     2010-01-01 12:00:00  1257.514052
3     2010-01-01 18:00:00   518.152013
4     2010-01-02 00:00:00   568.118041
...                   ...          ...
1184  2010-12-30 18:00:00   280.980845
1185  2010-12-31 00:00:00   195.742638
1186  2010-12-31 06:00:00   215.866280
1187  2010-12-31 12:00:00   164.523398
1188  2010-12-31 18:00:00   129.866575

[1189 rows x 2 columns]